In [1]:
!pip install polars
import pandas as pd
import subprocess
import tempfile

import polars as pl

splits = {
    'train': 'hf://datasets/blastwind/deprecated-github-code-haskell-function/data/train-*-of-*.parquet',
    'test': 'hf://datasets/blastwind/deprecated-github-code-haskell-function/data/test-*-of-*.parquet',
    'valid': 'hf://datasets/blastwind/deprecated-github-code-haskell-function/data/valid-00000-of-00001-636cb804972d8982.parquet'
}


df = pl.read_parquet(splits['train'])

dataframe = df.to_pandas()

dataframe = dataframe.loc[
    (dataframe["is_commented"] == True) & (dataframe["n_ast_nodes"] < 50),
    ["full_code", "uncommented_code", "function_only_code"]
]

dataframe = dataframe.head(100)




                                             full_code  \
11   -- 4 ^ 4 = 256\n\n-- 4\nprodTBool :: (Bool, Bo...   
22   -- | Uniquely identifies the type of this reso...   
24   -- | Creates a value of 'MapsPublish' with the...   
28   -- | Helper to print a value to lazy bytestrin...   
42   -- | Process the events.\nprocessEvents :: Eve...   
..                                                 ...   
841  ----------------------------------------------...   
843  -- | Let an expression remember its sugared fo...   
862  -- e :: ArrowComb (~>) => E (~>) ((), ()) ((),...   
874  -- | Is the Safe Haskell safe inference mode a...   
876  -- It is this declaration that causes nontermi...   

                                      uncommented_code  \
11   prodTBool :: (Bool, Bool, Bool)\nprodTBool = u...   
22   iKind :: Lens' Instance (Maybe Text)\niKind = ...   
24   mapsPublish\n    :: Text -- ^ 'mapId'\n    -> ...   
28   showb :: Show a => a -> BSL.ByteString\nshowb ...   
42   processE

In [3]:
dataframe = df.to_pandas()
dataframe["hlint_output"] = dataframe["function_only_code"].apply(run_hlint_on_snippet)
dataframe.loc[
    (dataframe["hlint_output"] != "No hints")
]

dataframe

KeyboardInterrupt: 

In [ ]:
def run_hlint_on_snippet(haskell_code):
    """Runs hlint on a single Haskell code snippet."""
    with tempfile.NamedTemporaryFile(suffix=".hs", mode="w", delete=True) as temp_file:
        temp_file.write(haskell_code)
        temp_file.flush()  
        try:
            result = subprocess.run(
                ["hlint", temp_file.name],
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True,
                check=True,
            )
            return result.stdout.strip()  # Return the output from hlint
        except subprocess.CalledProcessError as e:
            return f"Error: {e.stderr.strip()}"  # Return the error message

dataframe["hlint_output"] = dataframe["function_only_code"].apply(run_hlint_on_snippet)

print(dataframe)
